In [ ]:
# In this notebook, I'll leaverage the knowledge learned in cp5 to exploring my capstone project dataset.
# since my dataset is from different sources, I'll explore the various datasets one by one. 
# only sample one file from those different dataset sources.

In [200]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import xml.etree.ElementTree as ET

In [135]:
# some local env
# main dataset path
data_parent_path = './data/'

# ag_news
data_agnews = data_parent_path + 'ag_news_csv/train.csv'

# nyt article comments
data_nytcomments = data_parent_path + 'nyt_comments/CommentsApril2017.csv'
data_nytarticles = data_parent_path + 'nyt_comments/ArticlesApril2017.csv'

# one_week_newsfeed
data_oneweeknews = data_parent_path + 'one_week_newsfeeds/news-week-17aug24.csv'

# sogou news
data_sogou_news = data_parent_path + 'sogou_news_csv/train.csv'

# reuters
data_reuters = data_parent_path + 'reuters21578/reut2-000.sgm'

# social medial news
data_sns = data_parent_path + 'sns_dataset/News_Final.csv'

# twitter, 1,600,000 
data_tweets = data_parent_path + 'training.1600000.processed.noemoticon.csv'

### 1 - ag_news:
ag_news contains 120,000 rows, 3 columns, including classes, title, and content.
every columns is well orgnized, without any missing data.

In [32]:
# ag_news
ag_news_df = pd.read_csv(data_agnews, header=None)
ag_news_df.head()

,0,1,2
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [33]:
ag_news_df.columns

Int64Index([0, 1, 2], dtype='int64')

In [34]:
ag_news_df.columns = ['category', 'title', 'content']

In [35]:
ag_news_df.head(10)

,category,title,content
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
5,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...
6,3,Money Funds Fell in Latest Week (AP),AP - Assets of the nation's retail money marke...
7,3,Fed minutes show dissent over inflation (USATO...,USATODAY.com - Retail sales bounced back a bit...
8,3,Safety Net (Forbes.com),Forbes.com - After earning a PH.D. in Sociolog...
9,3,Wall St. Bears Claw Back Into the Black,"NEW YORK (Reuters) - Short-sellers, Wall Stre..."


In [36]:
ag_news_df.tail()

,category,title,content
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...
119999,2,Nets get Carter from Raptors,INDIANAPOLIS -- All-Star Vince Carter was trad...


In [37]:
ag_news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 3 columns):
category    120000 non-null int64
title       120000 non-null object
content     120000 non-null object
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [38]:
ag_news_df.describe()

,category
count,120000.000000
mean,2.500000
std,1.118039
min,1.000000
25%,1.750000
50%,2.500000
75%,3.250000
max,4.000000


In [39]:
ag_news_df.title.iloc[1]

'Carlyle Looks Toward Commercial Aerospace (Reuters)'

In [40]:
ag_news_df.content.iloc[1]

'Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'

### 2. nyt articles comments
* there are two different csv, including nyt-articiles, nyt-comments
* in nyt-articles, there are totale 16 columns and 886 rows in each csv files
* in nyt-articles , unfortunately, there is no real article content, instead it provide a link to the real articles.
* I define a function to extract real content from nyt website. It works

* in comment csv, there are 243832 entries, total 34 columns
* I can join these two table by using articleID, 
* commentBody is the comment column 

In [41]:
nyt_comments_df = pd.read_csv(data_nytcomments)
nyt_articles_df = pd.read_csv(data_nytarticles)

In [42]:
nyt_articles_df.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


In [43]:
nyt_articles_df.columns

Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [44]:
nyt_articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886 entries, 0 to 885
Data columns (total 16 columns):
abstract            22 non-null object
articleID           886 non-null object
articleWordCount    886 non-null int64
byline              886 non-null object
documentType        886 non-null object
headline            886 non-null object
keywords            886 non-null object
multimedia          886 non-null int64
newDesk             886 non-null object
printPage           886 non-null int64
pubDate             886 non-null object
sectionName         886 non-null object
snippet             886 non-null object
source              886 non-null object
typeOfMaterial      886 non-null object
webURL              886 non-null object
dtypes: int64(3), object(13)
memory usage: 110.9+ KB


In [45]:
nyt_articles_df.describe()

,articleWordCount,multimedia,printPage
count,886.000000,886.000000,886.000000
mean,1093.505643,3.857788,7.635440
std,799.095918,7.891072,10.809874
min,57.000000,0.000000,0.000000
25%,708.500000,3.000000,0.000000
50%,982.500000,3.000000,2.000000
75%,1291.750000,3.000000,13.000000
max,7832.000000,68.000000,74.000000


In [76]:
# since there is no real content, I have to extract it from the link.
# I'll define a function, and apply webLink as the input para, and for a content column.

def nyt_get_content(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text)

    lst = []
    for para in soup.select('.css-exrw3m'):
        lst += para.text
    return ''.join(lst)

In [88]:
# it requires lot time to run the whole df
# nyt_articles_df['content'] = nyt_articles_df.webURL.apply(nyt_get_content)
test = nyt_articles_df.webURL.iloc[0:2].apply(nyt_get_content)

In [90]:
print(test)

0    Adam Ferguson sensed the moment was right: “Ev...
1    These days, the last thing you want is to be k...
Name: webURL, dtype: object


In [91]:
nyt_articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886 entries, 0 to 885
Data columns (total 17 columns):
abstract            22 non-null object
articleID           886 non-null object
articleWordCount    886 non-null int64
byline              886 non-null object
documentType        886 non-null object
headline            886 non-null object
keywords            886 non-null object
multimedia          886 non-null int64
newDesk             886 non-null object
printPage           886 non-null int64
pubDate             886 non-null object
sectionName         886 non-null object
snippet             886 non-null object
source              886 non-null object
typeOfMaterial      886 non-null object
webURL              886 non-null object
content             886 non-null object
dtypes: int64(3), object(14)
memory usage: 117.8+ KB


In [92]:
nyt_comments_df.head()

,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,...,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,This project makes me happy to be a 30+ year T...,22022598.0,22022598,<br/>,comment,1.491237e+09,1,False,0.0,...,"Riverside, CA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
1,1491188619,Stunning photos and reportage. Infuriating tha...,22017350.0,22017350,NaN,comment,1.491180e+09,1,False,0.0,...,<br/>,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
2,1491188617,Brilliant work from conception to execution. I...,22017334.0,22017334,<br/>,comment,1.491179e+09,1,False,0.0,...,Raleigh NC,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
3,1491167820,NYT reporters should provide a contributor's l...,22015913.0,22015913,<br/>,comment,1.491150e+09,1,False,0.0,...,"Missouri, USA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
4,1491167815,Could only have been done in print. Stunning.,22015466.0,22015466,<br/>,comment,1.491147e+09,1,False,0.0,...,"Tucson, Arizona",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News


In [93]:
nyt_comments_df.describe()

,approveDate,commentID,commentSequence,createDate,depth,parentID,recommendations,recommendedFlag,replyCount,reportAbuseFlag,sharing,timespeople,trusted,updateDate,userID,inReplyTo,articleWordCount,printPage
count,2.438320e+05,2.438320e+05,2.438320e+05,2.438320e+05,243832.000000,2.438320e+05,243832.000000,0.0,243832.000000,0.0,243832.000000,243832.000000,243832.000000,2.438320e+05,2.438320e+05,2.438320e+05,243832.000000,243832.000000
mean,1.492504e+09,2.218861e+07,2.218861e+07,1.492495e+09,1.289425,6.416284e+06,17.625172,NaN,0.405734,NaN,0.086371,0.999643,0.033576,1.492506e+09,4.955750e+07,6.416194e+06,1293.667451,9.458652
std,1.330667e+06,1.855150e+05,1.855150e+05,1.328516e+06,0.453641,1.005529e+07,93.219419,NaN,1.944378,NaN,0.280912,0.018886,0.180136,1.330736e+06,2.338153e+07,1.005525e+07,946.593427,11.138473
min,1.491008e+09,2.199955e+07,2.199955e+07,1.491007e+09,1.000000,0.000000e+00,0.000000,NaN,-115.000000,NaN,0.000000,0.000000,0.000000,1.491009e+09,1.045000e+03,0.000000e+00,57.000000,0.000000
25%,1.491784e+09,2.209291e+07,2.209291e+07,1.491768e+09,1.000000,0.000000e+00,1.000000,NaN,0.000000,NaN,0.000000,1.000000,0.000000,1.491787e+09,3.219583e+07,0.000000e+00,828.000000,1.000000
50%,1.492437e+09,2.217668e+07,2.217668e+07,1.492428e+09,1.000000,0.000000e+00,3.000000,NaN,0.000000,NaN,0.000000,1.000000,0.000000,1.492437e+09,5.719876e+07,0.000000e+00,1124.000000,2.000000
75%,1.493120e+09,2.226047e+07,2.226047e+07,1.493087e+09,2.000000,2.205108e+07,9.000000,NaN,0.000000,NaN,0.000000,1.000000,0.000000,1.493122e+09,6.769995e+07,2.205108e+07,1385.000000,20.000000
max,1.524346e+09,2.682425e+07,2.682425e+07,1.524346e+09,3.000000,2.642620e+07,7938.000000,NaN,361.000000,NaN,1.000000,1.000000,1.000000,1.524346e+09,8.567808e+07,2.642620e+07,7832.000000,74.000000


In [94]:
nyt_comments_df.columns

Index(['approveDate', 'commentBody', 'commentID', 'commentSequence',
       'commentTitle', 'commentType', 'createDate', 'depth',
       'editorsSelection', 'parentID', 'parentUserDisplayName', 'permID',
       'picURL', 'recommendations', 'recommendedFlag', 'replyCount',
       'reportAbuseFlag', 'sharing', 'status', 'timespeople', 'trusted',
       'updateDate', 'userDisplayName', 'userID', 'userLocation', 'userTitle',
       'userURL', 'inReplyTo', 'articleID', 'sectionName', 'newDesk',
       'articleWordCount', 'printPage', 'typeOfMaterial'],
      dtype='object')

In [95]:
nyt_comments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243832 entries, 0 to 243831
Data columns (total 34 columns):
approveDate              243832 non-null int64
commentBody              243832 non-null object
commentID                243832 non-null float64
commentSequence          243832 non-null int64
commentTitle             228498 non-null object
commentType              243832 non-null object
createDate               243832 non-null float64
depth                    243832 non-null int64
editorsSelection         243832 non-null bool
parentID                 243832 non-null float64
parentUserDisplayName    70527 non-null object
permID                   243832 non-null object
picURL                   243832 non-null object
recommendations          243832 non-null float64
recommendedFlag          0 non-null float64
replyCount               243832 non-null float64
reportAbuseFlag          0 non-null float64
sharing                  243832 non-null int64
status                   243832 non

In [100]:
print(nyt_comments_df.commentBody.iloc[2])

Brilliant work from conception to execution. I've never seen anything like it.  As a paper of record it is important to leave a record of these and other peoples for whom no one else speaks.  Please keep humanizing what are otherwise bland statistics.  It matters. Time and again their stories sounded like mine, could have been mine.  We're about to see hordes of people far away experiencing utter devastation.  Empathy goes away unless there are names and stories, and pictures.  Links to helping at organizations, resources....  


In [101]:
nyt_comments_df.articleID.iloc[0]

'58def1347c459f24986d7c80'

In [102]:
nyt_articles_df.articleID.iloc[0]

'58def1347c459f24986d7c80'

### 3 one_week_newsfeeds
* I have two week csv, one is from 2017-Aug-24, another is 2018-Aug-24
* In 2017-Aug-24 csv, there are 1398431 e ntries, 
* almost no missing data, except headline_text
* But when I check the content, I find there is no content, instead of a source_url, I try to visit some of the url, I find that there are many news feeds in German, and some of them reveal 404 error code.
* Also there are no special pattern for extracting the content.
* I don't know how to use this data source.

In [103]:
data_oneweeknews_df = pd.read_csv(data_oneweeknews)

In [104]:
data_oneweeknews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1398431 entries, 0 to 1398430
Data columns (total 4 columns):
publish_time     1398431 non-null int64
feed_code        1398431 non-null object
source_url       1398431 non-null object
headline_text    1397447 non-null object
dtypes: int64(1), object(3)
memory usage: 42.7+ MB


In [105]:
data_oneweeknews_df.head()

,publish_time,feed_code,source_url,headline_text
0,201708240000,w3-adweek,http://www.adweek.com/digital/facebook-2017-gl...,Here Are the Details on Facebook's Global Part...
1,201708240000,w3-aktiencheck-de,http://www.aktiencheck.de/news/Artikel-Aktien_...,Aktien New York Schluss: Moderate Verluste nac...
2,201708240000,w3-aktiencheck-de,http://www.aktiencheck.de/news/Artikel-US_Boer...,US-Börsen im Minus - Zittern vor Notenbankertr...
3,201708240000,w3-auto-economictimes-indiatimes,http://auto.economictimes.indiatimes.com/news/...,Petrol & diesel on the rise post daily price r...
4,201708240000,w3-azdailysun,http://azdailysun.com/sports/football/could-de...,Could DeShone Kizer end Browns' history of QB ...


In [106]:
data_oneweeknews_df.describe()

,publish_time
count,1.398431e+06
mean,2.017083e+11
std,2.092834e+04
min,2.017082e+11
25%,2.017083e+11
50%,2.017083e+11
75%,2.017083e+11
max,2.017083e+11


In [113]:
url = data_oneweeknews_df.source_url
print(url.values)

['http://www.adweek.com/digital/facebook-2017-global-partner-summit/?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%253A%2Badweek%252Fthe-press%2B%2528Adweek%2BDigital%2529'
 'http://www.aktiencheck.de/news/Artikel-Aktien_New_York_Schluss_Moderate_Verluste_kleiner_Vortagsrally-8076835'
 'http://www.aktiencheck.de/news/Artikel-US_Boersen_Minus_Zittern_Notenbankertreffen-8076829'
 ...
 'http://www.wxii12.com/article/company-recalls-natural-dish-soap-that-may-contain-bacteria/12140943'
 'http://www.wz.de/lokales/duesseldorf/im-sommer-gerne-in-die-natur-an-nord-und-ostsee-1.2505628'
 'http://www.ynet.co.il/articles/0%2C7340%2CL-5010000%2C00.html']


### 4 Sogou News
* This dataset is really big, about 1.27G for train data. Having 449,999 entries
* Using chinese Pinyin, how to use that?
* No column name, I need to append it by myself.
* Does this kind of dataset can be mixed with other language in training my model?

In [116]:
data_sogou_news_df = pd.read_csv(data_sogou_news, header=None)

In [120]:
data_sogou_news_df.columns = ['class', 'title', 'content']

In [121]:
data_sogou_news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 3 columns):
class      450000 non-null int64
title      450000 non-null object
content    450000 non-null object
dtypes: int64(1), object(2)
memory usage: 10.3+ MB


In [122]:
data_sogou_news_df.head()

,class,title,content
0,4,2008 di4 qi1 jie4 qi1ng da3o guo2 ji4 che1 zha...,2008di4 qi1 jie4 qi1ng da3o guo2 ji4 che1 zha3...
1,4,zho1ng hua2 ju4n jie2 FRV ya4o shi,tu2 we2i zho1ng hua2 ju4n jie2 FRV ya4o shi .
2,3,ya2ng sho4u che2ng,/ 1 zha1ng \n wa3ng yi4 go1ng si1 ba3n qua2n ...
3,2,guo2 ta4i ju1n a1n : guo2 dia4n dia4n li4 ke3...,xi1n la4ng ti2 shi4 : be3n we2n shu3 yu2 ya2n ...
4,3,sho3u du1 yi1n le4 jia1 ci2 sha4n yi4 ya3n ...,te2ng xu4n yi1n yue4 xu4n :2008 nia2n 5 yue4 1...


In [123]:
data_sogou_news_df.content.iloc[0]

'2008di4 qi1 jie4 qi1ng da3o guo2 ji4 che1 zha3n yu2 15 ri4 za4i qi1ng da3o guo2 ji4 hui4 zha3n zho1ng xi1n she4ng da4 ka1i mu4 . be3n ci4 che1 zha3n jia1ng chi2 xu4 da4o be3n yue4 19 ri4 . ji1n nia2n qi1ng da3o guo2 ji4 che1 zha3n shi4 li4 nia2n da3o che2ng che1 zha3n gui1 mo2 zui4 da4 di2 yi1 ci4 , shi3 yo4ng lia3o qi1ng da3o guo2 ji4 hui4 zha3n zho1ng xi1n di2 qua2n bu4 shi4 ne4i wa4i zha3n gua3n . yi3 xia4 we2i xia4n cha3ng mo2 te4 tu2 pia4n .'

In [124]:
data_sogou_news_df.title.iloc[0]

'2008 di4 qi1 jie4 qi1ng da3o guo2 ji4 che1 zha3n me3i nv3 mo2 te4 '

### 5 sns_ataset
* total 93,239 rows.
* news_final.csv contains the sns news content.
* other csv files only contains SOCIAL FEEDBACK DATA

In [136]:
data_sns_df = pd.read_csv(data_sns)

In [138]:
data_sns_df.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
0,99248.0,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,0.000000,-0.053300,-1,-1,-1
1,10423.0,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,0.208333,-0.156386,-1,-1,-1
2,18828.0,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-0.425210,0.139754,-1,-1,-1
3,27788.0,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,0.000000,0.026064,-1,-1,-1
4,27789.0,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,0.000000,0.141084,-1,-1,-1


In [140]:
data_sns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93239 entries, 0 to 93238
Data columns (total 11 columns):
IDLink               93239 non-null float64
Title                93239 non-null object
Headline             93224 non-null object
Source               92960 non-null object
Topic                93239 non-null object
PublishDate          93239 non-null object
SentimentTitle       93239 non-null float64
SentimentHeadline    93239 non-null float64
Facebook             93239 non-null int64
GooglePlus           93239 non-null int64
LinkedIn             93239 non-null int64
dtypes: float64(3), int64(3), object(5)
memory usage: 7.8+ MB


In [141]:
data_sns_df.columns

Index(['IDLink', 'Title', 'Headline', 'Source', 'Topic', 'PublishDate',
       'SentimentTitle', 'SentimentHeadline', 'Facebook', 'GooglePlus',
       'LinkedIn'],
      dtype='object')

In [145]:
data_sns_df.Headline.iloc[2]

"Nouriel Roubini, NYU professor and chairman at Roubini Global Economics, explains why the global economy isn't facing the same conditions"

In [146]:
data_sns_df.Title.iloc[2]

'Nouriel Roubini: Global Economy Not Back to 2008'

### 6 training.1600000.processed.noemoticon
* total 1,600,000 rows, 6 columns
* column-target, is the emotional classes (0 = negative, 2 = neutral, 4 = positive ), there are half of the datasets is negative, others are positive. No neutral seeds
* dtaset is from 2009- Apirl, May, June

In [150]:
data_tweets_df = pd.read_csv(data_tweets, encoding='ISO-8859-1', header=None)

In [152]:
data_tweets_df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

In [153]:
data_tweets_df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [154]:
data_tweets_df.target.value_counts()

4    800000
0    800000
Name: target, dtype: int64

In [155]:
data_tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
target    1600000 non-null int64
ids       1600000 non-null int64
date      1600000 non-null object
flag      1600000 non-null object
user      1600000 non-null object
text      1600000 non-null object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [156]:
data_tweets_df.tail()

,target,ids,date,flag,user,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [187]:
month_map = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 
             'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': 10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(text):
    m = month_map[text[4:7]]
    d = text[8:10]
    y = text[-4:]
    hh = text[11:13]
    mm = text[14:16]
    ss = text[17:19]
    time_str = '{0:s}-{1:s}-{2:s} {3:s}:{4:s}:{5:s}'.format(y, m, d, hh, mm, ss)
    return datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S')

In [188]:
test = parse_clf_time('Tue Jun 16 08:40:50 PDT 2009')
print(test)

2009-06-16 08:40:50


In [193]:
data_tweets_df['timestamp'] = (data_tweets_df.date.apply(parse_clf_time))
data_tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 7 columns):
target       1600000 non-null int64
ids          1600000 non-null int64
date         1600000 non-null object
flag         1600000 non-null object
user         1600000 non-null object
text         1600000 non-null object
timestamp    1600000 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 85.4+ MB


In [195]:
data_tweets_df.timestamp.apply(datetime.date).value_counts()

2009-06-06    111676
2009-06-01    110290
2009-06-15    109781
2009-05-30    103673
2009-06-07     96350
2009-05-31     94588
2009-05-29     73827
2009-06-16     67980
2009-06-02     64192
2009-06-05     58757
2009-06-20     45364
2009-06-17     44012
2009-06-19     43136
2009-06-18     43004
2009-06-03     41588
2009-05-17     40154
2009-05-22     39074
2009-05-18     36469
2009-05-03     35333
2009-04-19     27469
2009-05-02     27434
2009-05-10     26029
2009-04-18     21754
2009-06-25     19694
2009-04-20     18460
2009-05-14     17460
2009-04-07     17311
2009-06-23     17002
2009-06-21     16360
2009-05-28     15903
2009-05-04     15481
2009-06-22     12009
2009-05-09     11739
2009-05-26     10778
2009-05-16      9146
2009-04-21      8587
2009-06-14      8272
2009-06-04      7842
2009-05-01      7716
2009-06-24      6299
2009-05-11      4186
2009-05-13      4066
2009-04-06      3360
2009-04-17      3084
2009-05-21      2132
2009-05-27       841
2009-05-25       169
2009-05-23   

### 7 reuters
* 

In [239]:
lst = []

with open(data_reuters, 'r') as reuter_file:
    lines = reuter_file.read()
    xml = BeautifulSoup(lines)
    bodys = xml.findAll('text')

    for body in bodys:
        title = body.find('title').text if body.find('title') else ''
        date = body.find('dateline').text if body.find('dateline') else ''
        text = body.text
        lst.append([title, date, text])

data_reuters_df = pd.DataFrame(lst, columns=['title', 'date', 'content'])
data_reuters_df


,title,date,content
0,BAHIA COCOA REVIEW,"SALVADOR, Feb 26 -","\nBAHIA COCOA REVIEW\n SALVADOR, Feb 26 - S..."
1,STANDARD OIL <SRD> TO FORM FINANCIAL UNIT,"CLEVELAND, Feb 26 -",\nSTANDARD OIL <SRD> TO FORM FINANCIAL UNIT\n ...
2,TEXAS COMMERCE BANCSHARES <TCB> FILES PLAN,"HOUSTON, Feb 26 -",\nTEXAS COMMERCE BANCSHARES <TCB> FILES PLAN\n...
3,TALKING POINT/BANKAMERICA <BAC> EQUITY OFFER,"LOS ANGELES, Feb 26 -",\nTALKING POINT/BANKAMERICA <BAC> EQUITY OFFER...
4,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,"WASHINGTON, Feb 26 -",\nNATIONAL AVERAGE PRICES FOR FARMER-OWNED RES...
...,...,...,...
995,ASHTON-TATE <TATE> TO OFFER COMMON SHARES,"TORRANCE, Calif., March 3 -",\nASHTON-TATE <TATE> TO OFFER COMMON SHARES\n ...
996,KEYCORP <KEY> REGISTERS SUBORDINATED NOTES,"ALBANY, N.Y., Marc 3 -",\nKEYCORP <KEY> REGISTERS SUBORDINATED NOTES\n...
997,<NATIONAL SEA PRODUCTS LTD> 4TH QTR NET,"HALIFAX, Nova Scotia, March 3 -\n",\n<NATIONAL SEA PRODUCTS LTD> 4TH QTR NET\n ...
998,U.K. MONEY MARKET SHORTAGE FORECAST REVISED DOWN,"LONDON, March 3 -",\nU.K. MONEY MARKET SHORTAGE FORECAST REVISED ...


In [240]:
data_reuters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
title      1000 non-null object
date       1000 non-null object
content    1000 non-null object
dtypes: object(3)
memory usage: 23.6+ KB
